In [25]:
import re
from datetime import datetime
import os
import MySQLdb as mariadb
from astropy.io import ascii
import numpy as np
from numpy.random import randint
import scipy.optimize
from astropy.table import vstack, Table

In [2]:
with open(os.getcwd()+'/19JAN02XA/History/19JAN02XA_V000_kMk4.hist') as file:
    contents = file.read()
    section = contents.split('+')
    
with open(os.getcwd()+'/skd_files/r1875.skd') as file:
    skd_contents = file.read()

In [46]:
# Testing on different experiments
with open(os.getcwd()+'/NEXT_VERSION_TESTING/corr_files/r1876.corr') as file:
    contents = file.read()
    section = contents.split('+')
    
with open(os.getcwd()+'/NEXT_VERSION_TESTING/skd_files/r1876.skd') as file:
    skd_contents = file.read()

In [47]:
def droppedChannels(text_section):
    station_id = [['KATH12M', 'YARRA12M', 'HOBART12', 'HOBART26'], ['Ke', 'Yg', 'Hb', 'Ho'], ['a', 'i', 'd', 'H']]
    dropped_chans = []
    for ant in station_id[1]:
        regex = ant + '.*'
        dropped = re.findall(regex,text_section,re.MULTILINE)
        if dropped == []:
            dropped_chans.append('')            
        elif len(dropped[0]) < 4:
            dropped_chans.append('')
        else:
            dropped_chans.append(dropped)
    return dropped_chans  
    # This function takes a block of text, and scrapes out whether any AuScope antennas have dropped channels
    # The input of this function is a text section from the correlator report (section[5])

In [48]:
droppedChannels(section[5])

['', '', '', '']

In [49]:
def manualPcal(text_section):
    station_id = [['KATH12M', 'YARRA12M', 'HOBART12', 'HOBART26'], ['Ke', 'Yg', 'Hb', 'Ho'], ['a', 'i', 'd', 'H']]
    manual_pcal = []
    for ant in station_id[1]:
        if ant in text_section:
            manual_pcal.append(True)
        else:
            manual_pcal.append(False)
    return manual_pcal
    # this determines whether manual pcal happened for any of our telescopes.
    # The input of this function is a text section from the correlator report (section[6])

In [50]:
manualPcal(section[6])

[False, False, False, False]

In [51]:
auscope_station_id = [['KATH12M', 'YARRA12M', 'HOBART12', 'HOBART26'], ['Ke', 'Yg', 'Hb', 'Ho'], ['a', 'i', 'd', 'H']]
sefd_stations = ['Hb','Ho','Ke','Yg', 'Ht', 'Is', 'Kk', 'Ma', 'Ny', 'On', 'Kv', 'Wn', 'Hh', 'Ft', 'Ts', 'Wm', 'Ww', 'Wa', 'Wz', 'Bd', 'Ag', 'Ys', 'Ur', 'Sy', 'Oh', 'Tc', 'Ai', 'Cc','Vm','Vs']

In [60]:
# creating a mask for only the rows that are relevant for auscope antennas
col_names = ['bl', 'X_snr', 'X_n', 'S_snr', 'S_n']
snr_data = ascii.read(section[10], data_start=4, fast_reader=True, names=col_names)
valid_rows = []
for i in range(0, len(snr_data[:]['bl'])):
    if any(char in snr_data[i]['bl'] for char in sefd_stations):
        valid_rows.append(i)
sefd_data = snr_data[valid_rows]

In [53]:
def sefdTableExtract(text_section):
    #sefd_stations = ['q', 'Q', 'a', 'K', 'I', 'N', 'X', 'j', 'W', 'i', 'B', 'b', 'd', 'H', 'T', 'E', 'u', 'V', 'Y', 'R']
    col_names = ['bl', 'X_snr', 'X_n', 'S_snr', 'S_n']
    snr_data = ascii.read(text_section, data_start=4, fast_reader=True, names=col_names)
    #valid_rows = []
    #for i in range(0, len(snr_data[:]['bl'])):
    #    if any(char in snr_data[i]['bl'] for char in sefd_stations):
    #        valid_rows.append(i)
    #sefd_data = snr_data[valid_rows]
    return snr_data
    # have moved the check for valid sefd stations into the baseline constant function
    
    # This extracts the sefd ratio table for any of the 20 telescopes defined in the sefd estimation function.
    # Make sure if you change which stations you want to be apart of the calculations, change this sefd_stations
    # array and the one in the sefd estimator function.

In [54]:
contents = section[4]
regex = '^\s{1}([^\s].*)'
antennas_corr_report = re.findall(regex,contents,re.MULTILINE)

antennas_corr_referenceOLD = []
for line in antennas_corr_report:
    line = line.split()
    ref = [line[0], line[1][1:3], line[1][4]]
    antennas_corr_referenceOLD.append(ref)

antennas_corr_referenceOLD

[['HART15M', 'Ht', 'q'],
 ['KATH12M', 'Ke', 'a'],
 ['KOKEE', 'Kk', 'K'],
 ['MATERA', 'Ma', 'I'],
 ['SEJONG', 'Kv', 'j'],
 ['WETTZ13N', 'Wn', 'W'],
 ['YARRA12M', 'Yg', 'i']]

In [55]:
contents = section[4]
regex = '\(.{4}\)'
test = re.findall(regex,contents,re.MULTILINE)

antennas_corr_reference = []
for line in test:
    if '/' in line:
        ref = [line[1:3],line[4]]
        antennas_corr_reference.append(ref)
    elif '-' in line:
        ref = [line[3:5], line[1]]
        antennas_corr_reference.append(ref)

antennas_corr_reference

[['Ht', 'q'],
 ['Ke', 'a'],
 ['Kk', 'K'],
 ['Ma', 'I'],
 ['Kv', 'j'],
 ['Wn', 'W'],
 ['Yg', 'i']]

In [16]:
def antennaReference_CORR(text_section):
    regex = '\(.{4}\)'
    antennas_corr_report = re.findall(regex,text_section,re.MULTILINE)
    antennas_corr_reference = []
    for line in antennas_corr_report:
        if '/' in line:
            ref = [line[1:3],line[4]]
            antennas_corr_reference.append(ref)
        elif '-' in line: # this is to handle some funky corr report styles.
            ref = [line[3:5], line[1]]
            antennas_corr_reference.append(ref)
    return antennas_corr_reference
    # This function takes the 4th section of the corr report and gives the 2 character
    # station code plus the single character corr code.

In [17]:
antennaReference_CORR(section[4])

[['Ht', 'q'],
 ['Ke', 'a'],
 ['Kk', 'K'],
 ['Ma', 'I'],
 ['Kv', 'j'],
 ['Wn', 'W'],
 ['Yg', 'i']]

In [56]:
regex = "^A\s\s.*"
alias_reference = re.findall(regex,skd_contents,re.MULTILINE)


antenna_reference = []
for entry in alias_reference:
    entry = entry.split()
    ref = [entry[2], entry[14], entry[15]]
    antenna_reference.append(ref)

print(antenna_reference)

[['HART15M', 'Ht', 'Ht'], ['KATH12M', 'Ke', 'Ke'], ['KOKEE', 'Kk', '102'], ['MATERA', 'Ma', '119'], ['SEJONG', 'Kv', 'Kv'], ['WETTZ13N', 'Wn', 'Wn'], ['YARRA12M', 'Yg', 'Yg']]


In [57]:
def antennaReference_SKD(text_section):
    regex = "^A\s\s.*"
    alias_reference = re.findall(regex,text_section,re.MULTILINE)
    antenna_reference = []
    for entry in alias_reference:
        entry = entry.split()
        ref = [entry[2], entry[14], entry[15]]
        antenna_reference.append(ref)
    return antenna_reference

In [58]:

regex_sefd = "^T\s{3}.*"
sefd_skd = re.findall(regex_sefd,skd_contents,re.MULTILINE)

stations_SEFD =[]
for line in sefd_skd:
    line = line.split()
    for i in range(0, len(antenna_reference)):
        if line[1] in antenna_reference[i]:
            SEFD_X_S = [antenna_reference[i][1], line[6], line[8]]
            stations_SEFD.append(SEFD_X_S)
print(stations_SEFD)

[['Ht', '1400', '1050'], ['Ke', '4800', '4800'], ['Kk', '2000', '750'], ['Ma', '1600', '2200'], ['Kv', '5000', '5000'], ['Wn', '1400', '1050'], ['Yg', '5800', '4300']]


In [59]:
def predictedSEFDextract(text_section, antenna_reference):
    regex_sefd = "^T\s{3}.*"
    sefd_skd = re.findall(regex_sefd,text_section,re.MULTILINE)
    stations_SEFD =[]
    for line in sefd_skd:
        line = line.split()
        for i in range(0, len(antenna_reference)):
            if line[1] in antenna_reference[i]:
                SEFD_X_S = [antenna_reference[i][1], line[6], line[8]]
                stations_SEFD.append(SEFD_X_S)
    return stations_SEFD
    # This block of code grabs all the SEFD setting lines and pulls the X and S SEFD for each station.
    # It returns a list with each element containing [StationCode, SEFD_X, SEFD_S]

In [22]:
bl_sefd_x = []
bl_sefd_s = []
bl_const_x = []
bl_const_s = []
for i1 in range(0, len(snr_data)):
    bl = snr_data[i1][0]
    sefd_x = []
    sefd_s = []
    for i2 in range(0, len(antennas_corr_reference)):
        if antennas_corr_reference[i2][1] in bl:
            for j in range(0, len(stations_SEFD)):
                if antennas_corr_reference[i2][0] in stations_SEFD[j][0]:
                    sefd_x.append(stations_SEFD[j][1])
                    sefd_s.append(stations_SEFD[j][2])
    bl_sefd_x.append(sefd_x)
    bl_sefd_s.append(sefd_s)
    if snr_data[i1][1] == 0:
        const_x = 0#float('NaN')
    else: 
        const_x = (int(sefd_x[0])*int(sefd_x[1]))/snr_data[i1][1]  
    if snr_data[i1][3] == 0:
        const_s = 0#float('NaN')
    else:
        const_s = (int(sefd_s[0])*int(sefd_s[1]))/snr_data[i1][3] 
    bl_const_x.append(const_x)
    bl_const_s.append(const_s)
            
# This nightmare spaghetti calculates the constant for each baseline combination in the SEFD equation
# This constant is just (SEFD_pred_1 * SEFD_pred_2)/r_snr_12

In [29]:
snr_data_stacked = vstack([snr_data,snr_data])
snr_data_stacked

bl,X_snr,X_n,S_snr,S_n
str2,float64,int64,float64,int64
qa,0.34,113,0.65,142
qK,0.0,0,0.0,0
qI,0.53,313,0.75,319
qj,1.42,82,0.69,76
qW,0.69,282,0.92,313
qi,1.05,235,1.01,235
aK,0.33,124,0.65,159
aI,0.23,32,0.54,66
aj,0.56,169,0.49,117


In [30]:
def baselineConstantsWeights(SNR_DATA, antennas_corr_reference, stations_SEFD):
    bl_sefd_x = []
    bl_sefd_s = []
    bl_const_x = []
    bl_const_s = []
    weights_x = []
    weights_s = []
    sefd_stations = ['Ke','Yg', 'Hb', 'Ho', 'Ht', 'Is', 'Kk', 'Ma', 'Ny', 'On', 'Kv', 'Wn', 'Hh', 'Ft', 'Ts', 'Wm', 'Ww', 'Wa', 'Wz', 'Bd', 'Ag', 'Ys', 'Ur', 'Sy', 'Oh', 'Tc', 'Ai', 'Cc','Vm','Vs']
    for i1 in range(0, len(SNR_DATA)):
        bl = list(SNR_DATA[i1][0])
        sefd_x = []
        sefd_s = []
        for i2 in range(0, len(antennas_corr_reference)):
            if antennas_corr_reference[i2][0] in sefd_stations and antennas_corr_reference[i2][1] in bl:
                for j in range(0, len(stations_SEFD)):
                    if antennas_corr_reference[i2][0] == stations_SEFD[j][0]: # probably a more elegant way to do this with list.index()
                        sefd_x.append(stations_SEFD[j][1])
                        sefd_s.append(stations_SEFD[j][2])
        if len(sefd_x) > 1: # this checks that 2 valid telescopes are in the bl
            bl_sefd_x.append(sefd_x)
            bl_sefd_s.append(sefd_s)
            if SNR_DATA[i1][1] == 0:
                const_x = 0#float('NaN')
            else: 
                const_x = (int(sefd_x[0])*int(sefd_x[1]))/SNR_DATA[i1][1]  
            if SNR_DATA[i1][3] == 0:
                const_s = 0#float('NaN')
            else:
                const_s = (int(sefd_s[0])*int(sefd_s[1]))/SNR_DATA[i1][3] 
            bl_const_x.append(const_x)
            bl_const_s.append(const_s)
            w_x = np.sqrt(SNR_DATA[i1][2])
            w_s = np.sqrt(SNR_DATA[i1][4])
            weights_x.append(w_x)
            weights_s.append(w_s)
    weights_x = np.asarray(weights_x)
    weights_s = np.asarray(weights_s)    
    return bl_const_x, bl_const_s, weights_x, weights_s
    
    # This nightmare spaghetti calculates the constant for each baseline combination in the SEFD equation
    # This constant is just (SEFD_pred_1 * SEFD_pred_2)/r_snr_12
    # also outputs the weightings for the SEFD estimation (sqrt(n)).
    # It is important that the sefd_station list is the same as the list of stations in the
    # sefd estimation system!

In [64]:
bl_const_x, bl_const_s, weights_x, weights_s = baselineConstantsWeights(snr_data, antennas_corr_reference, stations_SEFD)

In [65]:
def sefd_bl_equations(x, SNR_DATA):
    # need to set the function up for many of the potential telescopes in the IVS experiments
    # The extra variables have no effect if not in the schedule and will instead just have the initial guess value returned.
    q, Q, a, K, I, N, X, j, W, i, B, b, d, H, T, E, u, V, Y, R = x
    stations = [['q', q],['Q', Q], ['a', a], ['K', K], ['I', I], ['N', N],['X', X], ['j', j], ['W', W], ['i', i], ['B', B], ['b',b], ['d', d], ['H', H], ['T', T], ['E', E], ['u',u], ['V', V], ['Y', Y],['R', R]]    
    output = []
    for i in range(0, len(SNR_DATA)):
        ants = list(SNR_DATA[i][0])
        equations = []
        for j in range(0, len(stations)):
            if stations[j][0] in ants:
                equations.append(stations[j][1])
        multi = equations[0]*equations[1]
        output.append(multi)
    output_array = np.asarray(output)
    return output_array

def system(x,w,SNR_DATA,b):
    return w*(sefd_bl_equations(x, SNR_DATA)-b)


x0 = 999*np.ones(20)
x = scipy.optimize.leastsq(system, x0, args=(weights_s,snr_data,bl_const_s))[0]
print(x)

[1056.78059566  999.         7149.91186081  731.06504773 2833.06924189
  999.          999.         6900.33884805 1331.99144709 4546.7486793
  999.          999.          999.          999.          999.
  999.          999.          999.          999.          999.        ]


# Attempting to make this code work for all corr reports...

Going to try and define the baselines in terms of the two letter codes rather than the 1 letter codes that change depending on correlator.

This should be possible by just changing the stations list in the sefd_bl_equations() function to two letter codes that have a correpsonding x variables (x1, x2, ...), then using the antennas_corr_ref list to translate between the codes used for a particular correlator (e.g a, q, i etc.) into the two letter codes, which in turn will add the corresponding x variables into the system of equations.

In [72]:
def sefd_bl_equations(x, SNR_DATA, antennas_corr_reference):
    # need to set the function up for many of the potential telescopes in the IVS experiments
    # The extra variables have no effect if not in the schedule and will instead just have the initial guess value returned.
    x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30 = x
    #stations = [['Hb', x1],['Ho', x2], ['Ke', x3], ['Yg', x4], ['Ht', x5], ['Is', x6],['Kk', x7], ['Ma', x8], ['Ny', x9], ['On', x10], ['Kv', x11] , ['Wn', x12]]    
    station_str = ['Ke','Yg', 'Hb', 'Ho','Ht', 'Is', 'Kk', 'Ma', 'Ny', 'On', 'Kv', 'Wn', 'Hh', 'Ft', 'Ts', 'Wm', 'Ww', 'Wa', 'Wz', 'Bd', 'Ag', 'Ys', 'Ur', 'Sy', 'Oh', 'Tc', 'Ai', 'Cc','Vm','Vs']
    station_var = [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30]
    output = []
    for i in range(0, len(SNR_DATA)):
        ants = list(SNR_DATA[i][0])
        equations = []
        for j in range(0, len(antennas_corr_reference)):
            if antennas_corr_reference[j][1] in ants and antennas_corr_reference[j][0] in station_str:
                equations.append(station_var[station_str.index(antennas_corr_reference[j][0])])
        if len(equations) > 1:  
            multi = equations[0]*equations[1]
            output.append(multi)
    output_array = np.asarray(output)
    return output_array

def system(x,w,SNR_DATA,antennas_corr_reference,b):
    return w*(sefd_bl_equations(x, SNR_DATA, antennas_corr_reference)-b)


x0 = 999*np.ones(30)
x = scipy.optimize.leastsq(system, x0, args=(weights_s,snr_data_stacked,antennas_corr_reference,bl_const_s))[0]
print(x)

[7149.91186115 4546.74867895  999.          999.         1056.78059596
  999.          731.06504767 2833.06924242  999.          999.
 6900.33884803 1331.99144707  999.          999.          999.
  999.          999.          999.          999.          999.
  999.          999.          999.          999.          999.
  999.          999.          999.          999.          999.        ]


The above function now works for calculating the SEFD of any experiment in that 30 antenna list.

Need to make sure that only baselines with telescopes in that 30 get fed into this system.

In [76]:
def tableLengthHacker(SNR_DATA, antennas_corr_reference, stations_SEFD):
    # This function is purely to deal with the fact that my SEFD estimation system can't handle cases 
    # where there are less equations generated than the number of defined antennas (30). It simply
    # stacks the snr_data table so the generated equations are 30 or more. This does not do anything to the
    # values because every equation is just occuring N times, so the increase in weighting is equal. It simply
    # bypasses the shortcomings of doing this in python.
    bl_const_x, bl_const_s, weights_x, weights_s = baselineConstantsWeights(SNR_DATA, antennas_corr_reference, stations_SEFD)
    while len(bl_const_x) < 30:
        SNR_DATA = vstack([SNR_DATA,SNR_DATA])
        bl_const_x, bl_const_s, weights_x, weights_s = baselineConstantsWeights(SNR_DATA, antennas_corr_reference, stations_SEFD)
    return bl_const_x, bl_const_s, weights_x, weights_s, SNR_DATA

In [77]:
bl_const_x, bl_const_s, weights_x, weights_s, snr_data_stacked = tableLengthHacker(snr_data, antennas_corr_reference, stations_SEFD)

In [46]:
antennas_corr_reference#.index('Ht')
stations_str = ['Hb','Ho','Ke','Yg', 'Ht', 'Is', 'Kk', 'Ma', 'Ny', 'On', 'Kv', 'Wn']    



In [28]:
station_id = ["Ke", "Yg", "Hb", "Ho"]

In [76]:
sql_station = "UPDATE {} SET (estSEFD_X, estSEFD_S, Manual_Pcal, Dropped_Chans) VALUES (%s, %s, %s, %s) WHERE ExpID={};".format(station_id[0], 'R1875')
data = [9999.99, 10000.99, True, 'test test', 'r1875']

In [77]:
sql_station = """
    UPDATE {}
    SET estSEFD_X=%s, estSEFD_S=%s, Manual_Pcal=%s, Dropped_Chans=%s
    WHERE ExpID = %s
""".format(station_id[0])

In [78]:
sql_station


'\n    UPDATE Ke\n    SET estSEFD_X=%s, estSEFD_S=%s, Manual_Pcal=%s, Dropped_Chans=%s\n    WHERE ExpID = %s\n'

In [79]:
db_name = 'auscope_test2'
conn = mariadb.connect(user='auscope', passwd='password', db=str(db_name))
cursor = conn.cursor()
cursor.execute(sql_station, data)
conn.commit()